In [ ]:
"""

Created on Sun Jul 20 15:07:00 2020

@author: YanivMaimon

"""

#pre

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

############         create datas     ################


# first table's path
path = r'C:\Users\97250\Desktop\studied\R ,python\Datasets Kaggle\World Hapiness\WorldHappiness2015.csv'

### tow datas we will work with :

        #yearlyData            
        #timeData 

yearlyData =  pd.read_csv(path)
timeData = pd.read_csv(path)[['Country','Happiness Score']].set_index('Country')

In [ ]:
#       Add all happiness_Score to 'timeData'


for i in range(2015,2020):
    addPath = r'C:\Users\97250\Desktop\studied\R ,python\Datasets Kaggle\World Hapiness\WorldHappiness' + str(i) + '.csv'
    
    #try until columns are matching:
    try:
        try:
            addData = pd.read_csv(addPath)[['Country','Happiness Score']]
        except:
            addData = pd.read_csv(addPath)[['Country','Happiness.Score']]
    except:
        addData = pd.read_csv(addPath)[['Country or region','Score']]
        
    # Set unified form and join 
    addData.columns = ['Country','HappinessScore'+str(i)]
    addData = addData.set_index('Country')
    timeData = timeData.join(addData,how='inner')

#drop column of the initialized data
timeData.drop(timeData.columns[0],axis=1,inplace=True)

In [ ]:
#####           usefull functions       #######        


# agg count function   
def agg(colum , c='Happines Score',k =''):  ### d='r' will show relative data.
    
    """
    Aggregation function.
    
    Input:
        Colum - string or list of strings.
        c - string, kind of column to pick.
        k - string, kind of agg. .
        
    Return:
        DataFrame's aggregated form by given column(s).
    """
    
    #choose rank or score of happiness 
    if c == 'r':
        c='rank'
    
    a = yearlyData.groupby(colum)
    
    # These function change due to the data kind I work with.
    if k == 'v':
        a = a.var()
    else:
        a = a.mean()
        
    if type(colum) != list: # if one column has been given
        return a[c].sort_values(ascending=False)
    else:
        return a[[c]].unstack(0)
        

In [ ]:
dic = {'l':(24,18),'m':(16,12),'s':(12,8)}    
  
def saver(st):
    plt.savefig(r'C:\Users\97250\Desktop\studied\R ,python\Datasets Kaggle\Outputs data\WorldHappiness\'' + st + '.png')

In [ ]:
          
# visual plots 
def show(data,s = 'm',st='',save=False):
    """
    Plots Bar chart.
    input:
        data - DataFrame.
        st   - string - what to print in title.
        s    - string - Output size.
    """
    
    scale = dic[s]
    kind = 'line' if len(data.shape)==2 else 'bar' #len(data.shape)==2 means: 2 dimentional data
    
    data.plot(kind = kind,lw = (scale[0]//4),figsize = scale,fontsize = scale[0]+4)
    plt.title("\n " + st + ": \n",fontsize = 2*scale[0])
    plt.legend(loc='upper right',prop={'size':1.2*scale[0]})    
    if save:
        saver(st)

In [ ]:
# heatmap function
def heat(data,s='m',st='',save=False):
    
    scale = dic[s]
    
    sns.set(font_scale = 1.8)
    plt.figure(figsize = scale)
    sns.heatmap(data = data ,lw=1 ,linecolor = 'white',cmap = 'Reds', annot = True)
    plt.title('\n'+ st +': \n',fontsize = 2*scale[0])
    if save :
        saver(st)

In [ ]:
def topCountries(top=10):
    
    """
    This func return data for top countries throughout time.
    """
    
    #top 10 deadly countries
    countries = agg('country')[:top].index
    print(countries)
    #grab aggregated data for these countries
    dataOfTop10 = agg(['country']).query("country in @countries")
    #unstack data
    dataOfTop10 = dataOfTop10.unstack(1)
    #remove multiindexes
    dataOfTop10 = dataOfTop10.transpose().reset_index(level=0, drop=True).transpose()
    #sort by year
    dataOfTop10.sort_index(inplace=True)
    return dataOfTop10

In [ ]:
# do notice family column includes 'social-suupport column inside.


# show top countries through time

top10data = timeData.iloc[:10].transpose()
show(top10data,st ='top 10 happy countries through time' ,save=True)

################################


#this is an interesting one
#              lets find most varianted countries through years

#varianc ethrough years
yearVariance =  timeData.transpose().var()
#avg of variance
varianceAvg = round(yearVariance.mean(),5)
#hifhly varianced countries
countriesVarienced = yearVariance[yearVariance>0.7*varianceAvg].index
# data of variented countries
dataOfTop = yearlyData.query("Country in @countriesVarienced")
# agg. by region
mostVariencedRegions = dataOfTop.groupby('Region').mean()[['Happiness Rank']]
#sort
mostVariencedRegions = mostVariencedRegions.sort_values(by= 'Happiness Rank',ascending=False)

In [ ]:
######        Machine Learning Section    #######


## try to predictit the 'Happiness Rank' of 10% of countries

#       preperation

yearlyData.set_index('Country',inplace=True)

X = yearlyData.drop(['Region', 'Happiness Rank', 'Happiness Score'],axis=1)
y = yearlyData['Happiness Score']

#   scale

#from sklearn.preprocessing import MinMaxScaler

#cols = X.columns
#X = MinMaxScaler().fit_transform(X)
#X = pd.DataFrame(X,columns = cols)

In [ ]:
#########

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

########################




#       desicion tree model 

##from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor


regressor = DecisionTreeRegressor(random_state=0)
#cross_val_score(regressor, X_train, y_train, cv=10)

regressor.fit(X_train,y_train)

predictions = regressor.predict(X_test)

In [ ]:
#evaluation

from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np

MAE = round(mean_absolute_error(predictions,y_test),5)
print('The RMSE is: ' , round(np.sqrt(mean_squared_error(predictions,y_test)),5))
print('The MAE  is: ' , round(mean_absolute_error(predictions,y_test),5))


######################

#           random forest model 

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=600)

rfr.fit(X_train,y_train)

predictions = rfr.predict(X_test)